# Basic Usage

Available methods: 
<br>

Code: 
- `executeCode`

Terminal Command: 
- `executeCommand`
- `startCommandExecution`
- `getTask`
- `stopTask`

File Management: 
- `writeFiles`
- `listFiles`
- `readFiles`
- `removeFiles`

In [1]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json

code_client = CodeInterpreter('us-east-1')


## Run Code

In [2]:

language = "python"
code_to_execute = """
print("Hello World!!!")
"""

# Configure and Start the code interpreter session
session_id = code_client.start(
    # identifier="some_identifier", # The code interpreter sandbox identifier to use. default: "aws.codeinterpreter.v1"
    # name= None, # A name for this session. If not provided, a name will be generated using a UUID.
    # session_timeout_seconds=100 #  The timeout for the session in seconds. Default to 900.
)
print(f"session started: {session_id}")


# if code_client.start is not called, the `invoke` function will call `start` internally
response = code_client.invoke("executeCode", {
    "language": language,
    "code": code_to_execute
})

# Process and print the response
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

# Clean up and stop the code interpreter session
code_client.stop()


session started: 01K1PPWYE373Z99SS2A8ABW0TX
{
  "content": [
    {
      "type": "text",
      "text": "Hello World!!!"
    }
  ],
  "structuredContent": {
    "stdout": "Hello World!!!",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.014868497848510742
  },
  "isError": false
}


## Run Terminal Command

For example, we can create a custom Code Interpreter tool with an execution role to upload/download files from Amazon S3. This allows your code to interact with S3 buckets for storing and retrieving data.

https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/code-interpreter-s3-integration.html


#### Execute Command

Wait for completion before returning.


In [3]:
session_id = code_client.start()
command_to_execute = "echo 'Hello world!'"
response = code_client.invoke("executeCommand", {
    "command": command_to_execute,
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

code_client.stop()

{
  "content": [
    {
      "type": "text",
      "text": "Hello world!\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "Hello world!\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.004979133605957031
  },
  "isError": false
}


#### Start Command Execution

In [5]:
session_id = code_client.start()
command_to_execute = "sleep 30 && echo complete"
response = code_client.invoke("startCommandExecution", {
    "command": command_to_execute,
})

task_id = ""
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))
    task_id = event["result"]["structuredContent"]["taskId"]

# Not stopping the client yet


{
  "content": [
    {
      "type": "text",
      "text": "Successfully started a command execution task"
    }
  ],
  "structuredContent": {
    "taskId": "570c94a7-db4a-4b6e-ad7d-45fc722a3aee",
    "taskStatus": "submitted",
    "stdout": "",
    "stderr": ""
  },
  "isError": false
}


#### Get Execution task

possible task status: 'submitted'|'working'|'completed'|'canceled'|'failed'

In [6]:
# use the same session as above
response = code_client.invoke("getTask", {
    "taskId": task_id,
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

{
  "content": [
    {
      "type": "text",
      "text": "Task is working"
    }
  ],
  "structuredContent": {
    "taskStatus": "working",
    "stdout": "",
    "stderr": ""
  },
  "isError": false
}


#### Stop Execution Task

In [7]:
# use the same session as above
response = code_client.invoke("stopTask", {
    "taskId": task_id,
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

code_client.stop()

{
  "content": [
    {
      "type": "text",
      "text": "Successfully canceled a command execution task"
    }
  ],
  "isError": false
}


## File Management

In [8]:
# Some data Contents and code contents
data_file_content = "pikachu"
data_file_path = "data.txt"

# code with relative path
code_file_content = f"""
try:
    with open('{data_file_path}', 'r', encoding='utf-8') as content:
        content = content.read()
        print(content)
except FileNotFoundError:
    print(f"Error: The file '{data_file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {{e}}")
"""

code_file_path = "code.py"

#### Write (Create)
upload files to the Code Interpreter session

In [9]:
files_to_create = [
    {
        "path": data_file_path,
        "text": data_file_content
    },
    {
        "path": code_file_path,
        "text": code_file_content
    }
]

session_id = code_client.start()
response = code_client.invoke("writeFiles", {
    "content": files_to_create,
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

# NOT stopping the client yet!

{
  "content": [
    {
      "type": "text",
      "text": "Successfully wrote all 2 files"
    }
  ],
  "isError": false
}


#### Use Files

Run the code above that uses relative path

In [10]:
response = code_client.invoke("executeCode", {
    "language": "python",
    "code": code_file_content
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

{
  "content": [
    {
      "type": "text",
      "text": "pikachu"
    }
  ],
  "structuredContent": {
    "stdout": "pikachu",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.01548624038696289
  },
  "isError": false
}


#### List files

In [ ]:
# use the session from above
response = code_client.invoke("listFiles", {
    "path": "", # root
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))


{
  "content": [
    {
      "type": "resource_link",
      "uri": "file:///log",
      "name": "log",
      "description": "Directory"
    },
    {
      "type": "resource_link",
      "mimeType": "text/plain",
      "uri": "file:///data.txt",
      "name": "data.txt",
      "description": "File"
    },
    {
      "type": "resource_link",
      "uri": "file:///.ipython",
      "name": ".ipython",
      "description": "Directory"
    },
    {
      "type": "resource_link",
      "mimeType": "text/x-python",
      "uri": "file:///code.py",
      "name": "code.py",
      "description": "File"
    }
  ],
  "isError": false
}


#### Read Files

In [12]:
response = code_client.invoke("readFiles", {
    "paths": [data_file_path],
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))


{
  "content": [
    {
      "type": "resource",
      "resource": {
        "uri": "file:///data.txt",
        "mimeType": "text/plain",
        "text": "pikachu"
      }
    }
  ],
  "isError": false
}


#### Remove Files

In [13]:
response = code_client.invoke("removeFiles", {
    "paths": [code_file_path, data_file_path],
})
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

# stop the session after finish
code_client.stop()

{
  "content": [
    {
      "type": "text",
      "text": "Successfully removed all 2 files"
    }
  ],
  "isError": false
}
